In [16]:
import pandas as pd
import math
import get_df

In [15]:
df = get_df.get_df(nrows=500)[["match_id", "Svr", "1stIn", "2ndIn", "isSvrWinner"]]
df

,match_id,Svr,1stIn,2ndIn,isSvrWinner
0,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,0,0.0,0
1,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,1,NaN,1
2,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,0,1.0,0
3,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,1,NaN,1
4,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,1,1,NaN,1
...,...,...,...,...,...
495,20161112-W-Fed_Cup_World_Group_F-RR-Kristina_M...,2,0,1.0,1
496,20161112-W-Fed_Cup_World_Group_F-RR-Kristina_M...,2,1,NaN,1
497,20161112-W-Fed_Cup_World_Group_F-RR-Kristina_M...,2,0,0.0,0
498,20161112-W-Fed_Cup_World_Group_F-RR-Kristina_M...,2,0,1.0,0


In [3]:
df_fs = df[df["1stIn"] == 1]
df_ss = df[df["2ndIn"] == 1]

x1=df_fs.shape[0]/df.shape[0] #1st in / (number of games=number of 1st serves)

y1=df_fs[df_fs["isSvrWinner"]==1].shape[0]/df_fs.shape[0] #(number of games won on first serve/ (number of 1st in) )
w1=y1*x1

x2=df_ss.shape[0]/(df.shape[0]-df_fs.shape[0])
y2=df_ss[df_ss["isSvrWinner"]==1].shape[0]/df_ss.shape[0]
w2=y2*x2

p=w1+(1-x1)*w2

x1_var = math.sqrt(x1*(1-x1)/df.shape[0])
x2_var = math.sqrt(x2*(1-x2)/df.shape[0])
y1_var = math.sqrt(y1*(1-y1)/df_fs.shape[0])
y2_var = math.sqrt(y2*(1-y2)/df_fs.shape[0])
# w2_var=
print("x1:", x1)
print("y1:", y1)
print("w1:", w1)
print("x2:", x2)
print("y2:", y2)
print("w2:", w2)
print("P:", p)
print("real P:", df[df["isSvrWinner"]==1].shape[0]/df.shape[0])

x1: 0.6095
y1: 0.694011484823626
w1: 0.42300000000000004
x2: 0.8732394366197183
y2: 0.5513196480938416
w2: 0.48143405889884755
P: 0.611
real P: 0.611


In [4]:
#Demonstration of using the 1st and 2nd serve strategy twice
p_1s=w1+(1-x1)*w1
p_2s=w2+(1-x2)*w2
print("P using strategy 1:", p_1s) # Probability of winning using second serve strategy twice
print("P using strategy 2:", p_2s) # Probability of winning using first serve strategy twice

P using strategy 1: 0.5881815
P using strategy 2: 0.5424609114353212


In [19]:
# average of values by game

df_by_match = df.groupby(["match_id", "Svr"])
serve_stats = df_by_match.agg(x_1=("1stIn", "mean"), x_2=("2ndIn", "mean"))
serve_stats["y_1"] = df_by_match.apply(lambda x: x[x["1stIn"]==1]["isSvrWinner"].mean())
serve_stats["y_2"] = df_by_match.apply(lambda x: x[x["2ndIn"]==1]["isSvrWinner"].mean())

serve_stats["w_1"] = serve_stats["x_1"]*serve_stats["y_1"]
serve_stats["w_2"] = serve_stats["x_2"]*serve_stats["y_2"]

serve_stats



x_1       x_2  \
match_id                                           Svr                       
20161112-W-Fed_Cup_World_Group_F-RR-Kristina_Ml... 1    0.481928  0.837209   
                                                   2    0.518987  0.763158   
20161112-W-Fed_Cup_World_Group_F-RR-Petra_Kvito... 1    0.666667  0.652174   
                                                   2    0.584416  0.812500   
20161125-M-Davis_Cup_World_Group_F-RR-Ivo_Karlo... 1    0.535714  0.846154   
                                                   2    0.772727  1.000000   
20161126-M-Andria_CH-SF-Egor_Gerasimov-Matteo_B... 1    0.779661  0.769231   
                                                   2    0.617647  1.000000   
20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.635417  0.914286   
                                                   2    0.795699  0.894737   
20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    0.606250  0.968254   
                                                   2    0.582822  0.955882   
20221122-M-Davis_Cup_Finals-QF-Botic_Van_De_Zan... 1    0.535714  0.923077   
                                                   2    0.703704  0.875000   
20221122-M-Davis_Cup_Finals-QF-Jordan_Thompson-... 1    0.611570  0.978723   
                                                   2    0.569892  0.850000   
20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.472000  0.772727   
                                                   2    0.669811  0.914286   
20221124-W-Montevideo-R16-Brenda_Fruhvirtova-Ka... 1    0.764706  1.000000   
                                                   2    0.666667  0.833333   
20221126-W-Montevideo-SF-Diana_Shnaider-Kateryn... 1    0.636364  1.000000   
                                                   2    0.693878  1.000000   
20221127-W-Montevideo-F-Leolia_Jeanjean-Diana_S... 1    0.485294  0.800000   
                                                   2    0.615385  1.000000   
20221130-W-Andorra-R16-Sabine_Lisicki-Alina_Cha... 1    0.426230  0.914286   
                                                   2    0.673469  0.562500   
20221201-W-Andorra-R16-Cristina_Bucsa-Kateryna_... 1    0.608696  0.888889   
                                                   2    0.754098  0.600000   

                                                             y_1       y_2  
match_id                                           Svr                      
20161112-W-Fed_Cup_World_Group_F-RR-Kristina_Ml... 1    0.575000  0.638889  
                                                   2    0.756098  0.724138  
20161112-W-Fed_Cup_World_Group_F-RR-Petra_Kvito... 1    0.687500  0.466667  
                                                   2    0.666667  0.692308  
20161125-M-Davis_Cup_World_Group_F-RR-Ivo_Karlo... 1    1.000000  0.363636  
                                                   2    0.941176  0.800000  
20161126-M-Andria_CH-SF-Egor_Gerasimov-Matteo_B... 1    0.782609  0.800000  
                                                   2    0.857143  0.615385  
20161126-W-ITF_Valencia-SF-Jasmine_Paolini-Cris... 1    0.639344  0.468750  
                                                   2    0.527027  0.529412  
20161127-M-Davis_Cup_World_Group_F-RR-Marin_Cil... 1    0.793814  0.475410  
                                                   2    0.800000  0.476923  
20221122-M-Davis_Cup_Finals-QF-Botic_Van_De_Zan... 1    0.933333  0.750000  
                                                   2    0.736842  0.857143  
20221122-M-Davis_Cup_Finals-QF-Jordan_Thompson-... 1    0.716216  0.500000  
                                                   2    0.698113  0.588235  
20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pabl... 1    0.830508  0.529412  
                                                   2    0.633803  0.593750  
20221124-W-Montevideo-R16-Brenda_Fruhvirtova-Ka... 1    0.461538  0.500000  
                                                   2    0.416667  0.600000  
20221126-W-Montevideo-SF-D